In [3]:
import torch
import numpy as np
import scipy as sp
from numpy import ndarray 
from typing import Any, Callable, Dict, List, Optional, Tuple, Union, cast
import math
import scipy.constants


In [2]:
# RANDOM EXAMPLE
x = torch.tensor([1], requires_grad=True, dtype=torch.float)
y = torch.tensor([2], requires_grad=True, dtype=torch.float)

z = x**2 + y**3
z.backward()
print(x.grad)
print(y.grad)

tensor([2.])
tensor([12.])


In [4]:
#FUNCTIONS

#OPERATORS

def annihilation(dimension: int) -> ndarray:
    """
    Returns a dense matrix of size dimension x dimension representing the annihilation
    operator in number basis.
    """
    offdiag_elements = np.sqrt(range(1, dimension))
    return np.diagflat(offdiag_elements, 1)

def creation(dimension: int) -> ndarray:
    """
    Returns a dense matrix of size dimension x dimension representing the creation
    operator in number basis.
    """
    return annihilation(dimension).T




In [21]:
#MORE NOISE RELATED FUNCTIONS



#SOME GLOBAL PARAMETERS
R_k = sp.constants.h / (sp.constants.e**2.0)
R_0 = 50
T = 0.015
M = 400
Delta = 3.4e-4
x_qp = 3e-6

omega_low = 1e-9 * 2 * np.pi
omega_high = 3 * 2 * np.pi
t_exp = 1e4
A_flux = 1e-6
A_cc = 1e-7

#T1 CAPACITIVE
import scipy as sp
import scipy.constants

def q_cap_fun(omega) -> torch.Tensor:
    return (
        1e6
        * (2 * np.pi * 6e9 / torch.abs(omega)) ** 0.7
    )

def calc_therm_ratio(
    omega: float, T: float
) -> torch.Tensor:
    return (sp.constants.hbar * omega) / (sp.constants.k * T)

def spectral_density_cap(omega,EJ,EC,EL,flux):
    
    therm_ratio = calc_therm_ratio(omega, T)
    s = (
        2
        * 8
        * EC
        / q_cap_fun(omega)
        * (1 / torch.tanh(0.5 * torch.abs(therm_ratio)))
        / (1 + torch.exp(-therm_ratio))
    )
    s *= (
        2 * np.pi
    )  # We assume that system energies are given in units of frequency
    return s



In [6]:
#MORE NOISE RELATED FUNCTIONS

#T1 CHARGE IMPEDANCE

def spectral_density_ci(omega,EJ,EC,EL,flux):
    # Note, our definition of Q_c is different from Zhang et al (2020) by a
    # factor of 2

    Q_c = R_k / (8 * np.pi * complex(R_0).real)
    therm_ratio = calc_therm_ratio(omega, T)
    s = (
        2
        * ( omega / 1e9 ) #annoying unit stuff - gotta convert back to GHz
        / Q_c
        * (1 / torch.tanh(0.5 * therm_ratio))
        / (1 + torch.exp(-therm_ratio))
    )
    return s

# T1 FLUX BIAS LINE

def spectral_density_fbl(omega,EJ,EC,EL,flux):
    """
    Our definitions assume that the noise_op is dH/dflux.
    """

    therm_ratio = calc_therm_ratio(omega, T)

    s = (
        2
        * (2 * np.pi) ** 2
        * M**2
        * ( omega / 1e9 ) #annoying unit stuff - gotta convert back to GHz
        * sp.constants.hbar
        / complex(R_0).real
        * (1 / torch.tanh(0.5 * therm_ratio))
        / (1 + torch.exp(-therm_ratio))
    )
    # We assume that system energies are given in units of frequency and that
    # the noise operator to be used with this `spectral_density` is dH/dflux.
    # Hence we have to convert  2 powers of frequency to standard units
    s *= 1e9 ** 2.0
    return s



In [7]:
#T1 INDUCTIVE

def q_ind_fun(omega):

    therm_ratio = abs(calc_therm_ratio(omega, T))
    therm_ratio_500MHz = calc_therm_ratio(
        torch.tensor(2 * np.pi * 500e6), T
    )
    
    return (
        500e6
        * (
            torch.special.scaled_modified_bessel_k0(1 / 2 * therm_ratio_500MHz)
            * torch.sinh(1 / 2 * therm_ratio_500MHz)
            / torch.exp(1 / 2 * therm_ratio_500MHz)
        )
        / (
            torch.special.scaled_modified_bessel_k0(1 / 2 * therm_ratio)
            * torch.sinh(1 / 2 * therm_ratio)
            / torch.exp(1 / 2 * therm_ratio)
        )
    )

def spectral_density_ind(omega,EJ,EC,EL,flux):

    therm_ratio = calc_therm_ratio(omega,T)
    s = (
        2
        * EL
        / q_ind_fun(omega)
        * (1 / torch.tanh(0.5 * torch.abs(therm_ratio)))
        / (1 + torch.exp(-therm_ratio))
    )
    s *= (
        2 * np.pi
    )  # We assume that system energies are given in units of frequency
    return s

In [8]:
# T1 quasiparticle tunneling

def y_qp_fun(omega,EJ,EC,EL,flux):
    """
    Based on Eq. S23 in the appendix of Smith et al (2020).
    """
    # Note that y_qp_fun is always symmetric in omega, i.e. In Smith et al 2020,
    # we essentially have something proportional to sinh(omega)/omega
    omega = torch.abs(omega)
    Delta_in_Hz = Delta * sp.constants.e / sp.constants.h

    omega_in_Hz = omega / (2 * np.pi) 
    EJ_in_Hz = EJ * 1e9 # GHz to Hz

    therm_ratio = calc_therm_ratio(omega, T)
    Delta_over_T = calc_therm_ratio(
        2 * np.pi * Delta_in_Hz, T
    )

    re_y_qp = (
        np.sqrt(2 / np.pi)
        * (8 / R_k)
        * (EJ_in_Hz / Delta_in_Hz)
        * (2 * Delta_in_Hz / omega_in_Hz) ** (3 / 2)
        * x_qp
        * torch.sqrt(1 / 2 * therm_ratio)
        * torch.special.scaled_modified_bessel_k0(1 / 2 * torch.abs(therm_ratio))
        * torch.sinh(1 / 2 * therm_ratio)
        / torch.exp(1 / 2 * torch.abs(therm_ratio))
    )

    return re_y_qp

def spectral_density_qt(omega,EJ,EC,EL,flux):
    """Based on Eq. 19 in Smith et al (2020)."""
    therm_ratio = calc_therm_ratio(omega, T)

    return (
        2
        * omega / 1e9
        * complex(y_qp_fun(omega,EJ,EC,EL,flux)).real
        * (1 / torch.tanh(0.5 * therm_ratio))
        / (1 + torch.exp(-therm_ratio))
    )

In [9]:
#GENERAL T1 function

def t1(
        noise_op: torch.Tensor,
        spectral_density: Callable,
        eigvals: torch.Tensor,
        eigvecs: torch.Tensor,
        EJ: torch.Tensor,
        EC: torch.Tensor,
        EL: torch.Tensor,
        flux: torch.Tensor,
        i: int = 1,
        j: int = 0
    ):

        # We assume that the energies in `evals` are given in the units of frequency
        # and *not* angular frequency. The function `spectral_density` is assumed to
        # take as a parameter an angular frequency, hence we have to convert.


        ground = eigvecs[:,j]
        excited = eigvecs[:,i]
        ground_E = eigvals[j]
        excited_E = eigvals[i]

        omega = 2 * np.pi * (excited_E - ground_E) * 1e9

        s = (
            spectral_density(omega,EJ,EC,EL,flux) + spectral_density(-omega,EJ,EC,EL,flux)
        )
        
        rate = torch.matmul(noise_op,ground.unsqueeze(0).T)
        rate = torch.matmul(excited.unsqueeze(0).conj(),rate)
        rate = torch.abs(rate) **2 * s
        #print(torch.abs(torch.matmul(excited.unsqueeze(0).conj(),torch.matmul(noise_op,ground.unsqueeze(0).T))))

        return rate

In [10]:
# GENERAL TPHI FUNCTION

def tphi_1_over_f(
        A_noise: float,
        noise_op: torch.Tensor,
        eigvals: torch.Tensor,
        eigvecs: torch.Tensor,
        i: int = 1,
        j: int = 0
    ) -> float:

        ground = eigvecs[:,j]
        excited = eigvecs[:,i]
        ground_E = eigvals[j]
        excited_E = eigvals[i]

        rate = torch.abs(
            torch.matmul(ground.unsqueeze(0).conj(),torch.matmul(noise_op,ground.unsqueeze(0).T))
            - torch.matmul(excited.unsqueeze(0).conj(),torch.matmul(noise_op,excited.unsqueeze(0).T))
        )
        
        rate *= A_noise * np.sqrt(2 * np.abs(np.log(omega_low * t_exp)))

        # We assume that the system energies are given in units of frequency and
        # not the angular frequency, hence we have to multiply by `2\pi`
        rate *= 2 * np.pi

        return rate

In [11]:
# T1 FOR FLUXONIUM
def t1_effective (
    EJ: torch.Tensor,
    EC: torch.Tensor,
    EL: torch.Tensor,
    flux: torch.Tensor,
    dim: float,
):
    
    # PART 1 - GET HAMILTONIAN
    plasma_energy = math.sqrt(8.0*EL*EC)
    diag_elements = [(i + 0.5) for i in range(dim)]
    lc_osc = torch.tensor(np.diag(diag_elements),dtype=torch.double)
    lc_osc = lc_osc * plasma_energy

    phi_osc = (8.0 * EC / EL) ** 0.25
    phi = (
            (torch.tensor(creation(dim),dtype=torch.double) + torch.tensor(annihilation(dim),dtype=torch.double))
            * phi_osc/ math.sqrt(2)
        )

    argument = phi + 2 * np.pi * flux * torch.tensor(np.eye(dim),dtype=torch.double)

    cos_phi = (torch.linalg.matrix_exp(argument*1j)+torch.linalg.matrix_exp(argument*-1j))/2
    sin_phi = (torch.linalg.matrix_exp(argument*1j)-torch.linalg.matrix_exp(argument*-1j))/2j

    n_op = (
                1j
                * (torch.tensor(creation(dim),dtype=torch.double) - torch.tensor(annihilation(dim),dtype=torch.double))
                / (phi_osc * math.sqrt(2))
            )

    d_ham_d_flux = -2 * np.pi * EJ * sin_phi

    H = lc_osc - EJ*cos_phi
    #H = torch.triu(H, diagonal=1).T + torch.triu(H)
    #H.to(torch.double)

    #PART 2 - Get eigenvalues

    eigvals,eigvecs = torch.linalg.eigh(H,UPLO='U')

    # PART 3 - Find rate

    t1_rate = torch.zeros([1,1],dtype=torch.double)

    #3.1 t1_capacitive

    t1_rate += t1(n_op, spectral_density_cap, eigvals, eigvecs, EJ, EC, EL, flux)

    #3.2 t1_charge_impedance

    #t1_rate += t1(n_op, spectral_density_ci, eigvals, eigvecs, EJ, EC, EL, flux)

    #3.3 t1_flux_bias_line

    t1_rate += t1(d_ham_d_flux, spectral_density_fbl, eigvals, eigvecs, EJ, EC, EL, flux)

    #3.4 t1_inductive

    t1_rate += t1(phi.to(torch.cdouble), spectral_density_ind, eigvals, eigvecs, EJ, EC, EL, flux)

    #3.5 t1_quasiparticle_tunneling - a little dubious

    qt_argument = argument/2
    qt_noise = (torch.linalg.matrix_exp(qt_argument*1j)-torch.linalg.matrix_exp(qt_argument*-1j))/2j

    t1_rate += t1(qt_noise.to(torch.cdouble), spectral_density_qt, eigvals, eigvecs, EJ, EC, EL, flux)

    # PART 4 - Get time

    T1 = 1/t1_rate

    return T1

In [28]:
# T2 FOR FLUXONIUM
def t2_effective (
    EJ: torch.Tensor,
    EC: torch.Tensor,
    EL: torch.Tensor,
    flux: torch.Tensor,
    dim: float,
):
    
    # PART 1 - GET HAMILTONIAN
    plasma_energy = math.sqrt(8.0*EL*EC)
    diag_elements = [(i + 0.5) for i in range(dim)]
    lc_osc = torch.tensor(np.diag(diag_elements),dtype=torch.double)
    lc_osc = lc_osc * plasma_energy

    phi_osc = (8.0 * EC / EL) ** 0.25
    phi = (
            (torch.tensor(creation(dim),dtype=torch.double) + torch.tensor(annihilation(dim),dtype=torch.double))
            * phi_osc/ math.sqrt(2)
        )

    argument = phi + 2 * np.pi * flux * torch.tensor(np.eye(dim),dtype=torch.double)

    cos_phi = (torch.linalg.matrix_exp(argument*1j)+torch.linalg.matrix_exp(argument*-1j))/2
    sin_phi = (torch.linalg.matrix_exp(argument*1j)-torch.linalg.matrix_exp(argument*-1j))/2j

    n_op = (
                1j
                * (torch.tensor(creation(dim),dtype=torch.double) - torch.tensor(annihilation(dim),dtype=torch.double))
                / (phi_osc * math.sqrt(2))
            )

    d_ham_d_flux = -2 * np.pi * EJ * sin_phi
    d_ham_d_EJ = - cos_phi


    H = lc_osc - EJ*cos_phi
    #H = torch.triu(H, diagonal=1).T + torch.triu(H)
    #H.to(torch.double)

    #PART 2 - Get eigenvalues

    eigvals,eigvecs = torch.linalg.eigh(H,UPLO='U')

    # PART 3 - Find rate from T1

    t2_rate = torch.zeros([1,1],dtype=torch.double)

    #3.1 t1_capacitive

    t2_rate += 0.5 * t1(n_op, spectral_density_cap, eigvals, eigvecs, EJ, EC, EL, flux)

    #3.2 t1_charge_impedance

    #t2_rate += 0.5 * t1(n_op, spectral_density_ci, eigvals, eigvecs, EJ, EC, EL, flux)

    #3.3 t1_flux_bias_line

    t2_rate += 0.5 * t1(d_ham_d_flux, spectral_density_fbl, eigvals, eigvecs, EJ, EC, EL, flux)

    #3.4 t1_inductive

    t2_rate += 0.5 * t1(phi.to(torch.cdouble), spectral_density_ind, eigvals, eigvecs, EJ, EC, EL, flux)

    #3.5 t1_quasiparticle_tunneling - a little dubious

    qt_argument = argument/2
    qt_noise = (torch.linalg.matrix_exp(qt_argument*1j)-torch.linalg.matrix_exp(qt_argument*-1j))/2j

    t2_rate += 0.5 * t1(qt_noise.to(torch.cdouble), spectral_density_qt, eigvals, eigvecs, EJ, EC, EL, flux)

    # PART 4 - Find rate from Tphi

    #4.1 tphi_1_over_f_flux

    t2_rate += tphi_1_over_f(A_flux, d_ham_d_flux, eigvals, eigvecs)
    
    #4.2 tphi_1_over_f_cc

    t2_rate += tphi_1_over_f(A_cc, d_ham_d_EJ, eigvals, eigvecs)

    # PART 5 - Get time

    T2 = 1/t2_rate

    return T2

In [14]:
#TENSORS

# INPUT

#default values
EJ = torch.tensor([8.9], requires_grad=True, dtype=torch.double)
EC = torch.tensor([2.5], requires_grad=True, dtype=torch.double)
EL = torch.tensor([0.5], requires_grad=True, dtype=torch.double)
flux = torch.tensor([0.5], requires_grad=True, dtype=torch.double)

#random values
#EJ = torch.rand(1, requires_grad=True, dtype=torch.double)
#EC = torch.rand(1, requires_grad=True, dtype=torch.double)
#EL = torch.rand(1, requires_grad=True, dtype=torch.double)
#flux = torch.rand(1, requires_grad=True, dtype=torch.double)

#EJ.data = EJ.data * 100
#EC.data = EC.data * 100
#EL.data = EL.data * 100

# PARAMETERS

dim = 110

convergence_ratio = 1e-4
learn_rate = 5e3

# BOUNDS

EJ_bounds = [1e-3, 100]
EC_bounds = [1e-3, 100]
EL_bounds = [1e-3, 100]
flux_bounds = [0,1]

# GRADIENT DESCENT

max_T1 = 0

for i in range(0,10000):

    '''
    I haven't poked around with this too much, so 1/T1^2 is my current cost function.
    I'm sure there's a much much better choice, though, which I'll want to look into.
    '''
    T1 = t1_effective(EJ,EC,EL,flux,dim)

    if T1 > max_T1:
        max_T1 = T1
    
    cost = 1/(T1 **(1/2)) 
    cost.backward()

    print("CYCLE: ",i)
    print("T1 = ",T1.item(), ", cost = ", cost.item(), ", Best T1 = ", max_T1.item())
    print("EJ,EC,EL,flux = ",EJ.item(),EC.item(),EL.item(),flux.item())
    print("change EJ,EC,EL,flux = ",EJ.grad.item()*learn_rate,EC.grad.item()*learn_rate,EL.grad.item()*learn_rate,flux.grad.item()*learn_rate)
    
    # Near the boundary, we project the gradient onto it. This is the typical approach to constraints.
    if EJ.data - EJ.grad.data*learn_rate > EJ_bounds[0] and EJ.data - EJ.grad.data*learn_rate < EJ_bounds[1]:
        EJ.data = EJ.data - EJ.grad.data*learn_rate
    
    if EC.data - EC.grad.data*learn_rate > EC_bounds[0] and EC.data - EC.grad.data*learn_rate < EC_bounds[1]:
        EC.data = EC.data - EC.grad.data*learn_rate
    
    if EL.data - EL.grad.data*learn_rate > EL_bounds[0] and EL.data - EL.grad.data*learn_rate < EL_bounds[1]:
        EL.data = EL.data - EL.grad.data*learn_rate

    if flux.data - flux.grad.data*learn_rate > flux_bounds[0] and flux.data - flux.grad.data*learn_rate < flux_bounds[1]:
        flux.data = flux.data - flux.grad.data*learn_rate

    EJ.grad.data.zero_()
    EC.grad.data.zero_()
    EL.grad.data.zero_()
    flux.grad.data.zero_()


CYCLE:  0
T1 =  2602877.406472942 , cost =  0.0006198307860345488 , Best T1 =  2602877.406472942
EJ,EC,EL,flux =  8.9 2.5 0.5 0.5
change EJ,EC,EL,flux =  -0.22919655657446025 0.4685303742574976 0.6827903898409262 -3.5098920608631215e-12
CYCLE:  1
T1 =  2671429.2817181437 , cost =  0.0006118263244364951 , Best T1 =  2671429.2817181437
EJ,EC,EL,flux =  9.12919655657446 2.0314696257425022 0.5 0.5000000000035099
change EJ,EC,EL,flux =  -0.04109663088036333 0.17788165628686137 2.264392788796507 -8.02486528196211e-08
CYCLE:  2
T1 =  2496301.486252242 , cost =  0.0006329238808521665 , Best T1 =  2671429.2817181437
EJ,EC,EL,flux =  9.170293187454822 1.8535879694556407 0.5 0.5000000802521627
change EJ,EC,EL,flux =  0.0045050927849368295 0.09510973278105064 2.7427207077377154 -0.0035362913707410633
CYCLE:  3
T1 =  3052366.0085756113 , cost =  0.0005723763695819693 , Best T1 =  3052366.0085756113
EJ,EC,EL,flux =  9.165788094669885 1.7584782366745901 0.5 0.5035363716229038
change EJ,EC,EL,flux =  

KeyboardInterrupt: 

In [ ]:
# OTHER GRADIENT-BASED METHODS

In [29]:
import scqubits as scq
import scqubits.core.noise as noise
import scqubits.core.units as units
import numpy as np
import torch
from numpy import ndarray
import math

EJ = torch.tensor([8.9], requires_grad=True, dtype=torch.double)
EC = torch.tensor([2.5], requires_grad=True, dtype=torch.double)
EL = torch.tensor([0.5], requires_grad=True, dtype=torch.double)
flux = torch.tensor([0.5], requires_grad=True, dtype=torch.double)

print(t2_effective(EJ,EC,EL,flux,120))


fluxonium = scq.Fluxonium(EJ = 8.9,
                               EC = 2.5,
                               EL = 0.5,
                               flux = 0.5,
                               cutoff = 110)

print(fluxonium.t2_effective(get_rate=False))


tensor([[2362447.5933]], dtype=torch.float64, grad_fn=<MulBackward0>)
2362447.5215063957
